In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,InputLayer

In [3]:
df = pd.read_csv('../dataset/gold.csv')
dataset_length = len(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2000-01-04,281.0,281.0,281.00,282.7,4,USD
1,2000-01-05,283.2,283.2,283.20,281.1,16,USD
2,2000-01-06,281.4,281.4,281.40,281.4,0,USD
3,2000-01-07,281.9,281.9,281.90,281.9,0,USD
4,2000-01-10,281.7,281.7,281.70,281.7,0,USD
...,...,...,...,...,...,...,...
5757,2022-06-29,1821.4,1834.9,1810.70,1817.5,155542,USD
5758,2022-06-30,1819.2,1826.8,1802.50,1807.3,208898,USD
5759,2022-07-01,1808.1,1813.6,1783.40,1801.5,249489,USD
5760,2022-07-03,1812.5,1814.2,1809.05,1809.1,0,USD


# DATA PREPROCESSING : 
### Making it per month :

In [8]:
data = df.filter(['Close'])
dataset = data.values

import math
training_data_len = math.ceil(len(dataset) *.8)
training_data_len

4610

In [10]:
#scale the data :
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

In [16]:
#create training dataset :
WINDOW = 60
#create the sclaed training dataset
train_data = scaled_data[:training_data_len,:]
#split the data into x_train and y_train datasets
x_train = []
y_train = []

for i in range(WINDOW,len(train_data)):
    x_train.append(train_data[i-WINDOW:i,0])
    y_train.append(train_data[i,0])



(4550, 60) (4550,)


In [17]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [22]:
#reshape :
x_train = np.reshape(x_train , (x_train.shape[0],x_train.shape[1],1))
x_train.shape

(4550, 60, 1)

# Model creation

In [23]:
model = Sequential()

model.add(LSTM(50,return_sequences=True,input_shape = (WINDOW,1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))


model.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mean_absolute_error']
)

In [25]:
## callbacks
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=20, min_lr=0.00001)

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=1,
    epochs=100,
    callbacks=[reduce_lr]
)

In [31]:
#create testing data set :
test_data = scaled_data[training_data_len:,:]

#splitting :
x_test, y_test = [],[]

for i in range(WINDOW,len(test_data)):
    x_test.append(test_data[i-WINDOW:i,0])
    y_test.append(test_data[i,0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
y_test = np.array(y_test)
print(f"x_test shape = {x_test.shape} , y_test shape = {y_test.shape}")

x_test shape = (1092, 60, 1) , y_test shape = (1092,)
